In [1]:
"""
Summaries of USFS TreeMap linked to FIA plot data
Emphasis on 
    - Metrics of forest composition
    - Ecological gradients of species dominance
    - Forest structure (basal area, QMD, TPA, etc.)

Aggregate these statistics to FRP gridcells.

Author: maxwell.cook@colorado.edu
"""

import os, sys, time
import pandas as pd
import rioxarray as rxr
import geopandas as gpd

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready to go !")

Ready to go !


In [2]:
# Load the TreeMap (ca. 2016)
# Pixel values here denote the FIA plot ID ("tm_id")
fp = os.path.join(maindir,'data/landcover/USFS/RDS_TreeMap/TreeMap2016.tif')
treemap_da = rxr.open_rasterio(fp, masked=True, cache=False, chunks='auto').squeeze()
# Grab some raster metadata
shp, gt, wkt, nd = treemap_da.shape, treemap_da.spatial_ref.GeoTransform, treemap_da.rio.crs, treemap_da.rio.nodata
print(
    f"Shape: {shp}; \n"
    f"GeoTransform: {gt}; \n"
    f"WKT: {wkt}; \n"
    f"NoData Value: {nd}; \n"
    f"Data Type: {treemap_da[0].dtype}")
gc.collect() # clean up

Shape: (97383, 154221); 
GeoTransform: -2362845.0 30.0 0.0 3180555.0 0.0 -30.0; 
WKT: EPSG:5070; 
NoData Value: nan; 
Data Type: float64


60

In [3]:
# load and prepare our study region for cropping TreeMap
# Southern Rockies ecoregion bounds (buffered)
fp = os.path.join(projdir,'data/spatial/raw/boundaries/na_cec_eco_l3_srme.gpkg')
srm = gpd.read_file(fp)
# Crop the raster by the SRM bounds
bounds = srm.total_bounds # total bounds of ecoregion
treemap_da_c = treemap_da.rio.clip_box(
    minx=bounds[0]+10000, # +10km buffer
    miny=bounds[1]+10000, 
    maxx=bounds[2]+10000, 
    maxy=bounds[3]+10000
)
print(f"Cropped TreeMap to SRM bounds w/ 10km buffer.")
del treemap_da, bounds
gc.collect() # clean up

Cropped TreeMap to SRM bounds w/ 10km buffer.


54

In [4]:
# load the aggregated FRP grid
fp = os.path.join(projdir,'data/spatial/mod/VIIRS/viirs_snpp_jpss1_afd_latlon_fires_pixar_gridstats.gpkg')
grid = gpd.read_file(fp)
grid.columns

Index(['grid_index', 'grid_area', 'afd_count', 'unique_days', 'overlap',
       'frp_csum', 'frp_max', 'frp_min', 'frp_mean', 'frp_p90', 'frp_first',
       'day_max_frp', 'dt_max_frp', 'first_obs_date', 'last_obs_date',
       't4_max', 't4_mean', 't5_max', 't5_mean', 'day_count', 'night_count',
       'frp_max_day', 'frp_max_night', 'frp_csum_day', 'frp_csum_night',
       'frp_mean_day', 'frp_mean_night', 'frp_p90_day', 'frp_p90_night',
       'frp_first_day', 'frp_first_night', 'Fire_ID', 'Fire_Name', 'geometry'],
      dtype='object')

In [ ]:
# get the count/proportion of unique "tm_id" from TreeMap in grids
t0 = time.time()

# see __functions.py
grid_tmid = compute_band_stats(grid, treemap_da_c, 'grid_index', attr='tm_id')

t1 = (time.time() - t0) / 60
print(f"Total elapsed time: {t1:.2f} minutes.")
print("\n~~~~~~~~~~\n")

In [ ]:
# tidy columns
grid_tmid['count'] = grid_tmid['count'].astype(int)
grid_tmid['total_pixels'] = grid_tmid['total_pixels'].astype(int)
grid_tmid.rename(columns={'total_pixels': 'forest_pixels'}, inplace=True)
grid_tmid.head()

In [ ]:
# load the Tree Table
fp = os.path.join(maindir,'data/landcover/USFS/RDS_TreeMap/TreeMap2016_tree_table.csv')
tree_tbl = pd.read_csv(fp)
tree_tbl.columns

In [ ]:
# join to the grid data
grid_trees = grid_tmid.merge(tree_tbl, on='tm_id', how='left')
grid_trees.head()

In [ ]:
# identify the dominant forest species for each "tm_id"
spp_dominance = (
    grid_trees.groupby(['tm_id', 'COMMON_NAME'])['TREE']  # Sum tree counts (or use another metric)
    .sum()
    .reset_index()
    .sort_values(['tm_id', 'TREE'], ascending=[True, False])
    .drop_duplicates('tm_id')  # Keep only the top species per tm_id
    .rename(columns={'COMMON_NAME': 'MajoritySpp'})
)

# join back to the tm_id summary
grid_tmid_spp = grid_tmid.merge(spp_dominance[['tm_id', 'MajoritySpp']], on='tm_id', how='left')

# identify the landscape proportion of dominant species
spp_pr = (
    grid_tmid_spp.groupby('MajoritySpp')['count']
    .sum()
    .reset_index()
    .rename(columns={'count': 'maj_spp_count'})
    .sort_values(by='maj_spp_count', ascending=False)
)

# calculate the fraction
spp_pr['fraction'] = spp_pr['maj_spp_count'] / spp_pr['maj_spp_count'].sum()
spp_pr = spp_pr.sort_values(by='fraction', ascending=False)

# Identify species contributing 97% of the burned area
spp_pr['c_fraction'] = spp_pr['fraction'].cumsum()
top_species = spp_pr[spp_pr['c_fraction'] <= 0.99]
print(f"\nSpecies contributing to 97% of the burned area:\n{top_species}\n")

In [ ]:
# do some species regrouping

# pinon-juniper
pj = grid_trees[grid_trees['COMMON_NAME'].str.contains('pinyon|juniper', case=False, na=False)]
print(pj['COMMON_NAME'].unique())
# replace the species names
spp_remap = {name: 'pinon-juniper' for name in pj['COMMON_NAME'].unique()}

# spruce-fir
spruce_fir = grid_trees[grid_trees['COMMON_NAME'].str.contains(' fir|spruce', case=False, na=False)]
print(spruce_fir['COMMON_NAME'].unique())
# replace the species names
spp_remap.update({name: 'spruce-fir' for name in spruce_fir['COMMON_NAME'].unique()})

# oak woodland
oak = grid_trees[grid_trees['COMMON_NAME'].str.contains('oak|mountain-mahogany', case=False, na=False)]
print(oak['COMMON_NAME'].unique())
# replace the species names
spp_remap.update({name: 'oak-scrub' for name in spruce_fir['COMMON_NAME'].unique()})

# apply the new mapping
grid_tree_r = grid_trees.copy()
grid_tree_r['ForestType'] = grid_tree_r['COMMON_NAME'].map(spp_remap).fillna(grid_tree_r['COMMON_NAME'])

In [ ]:
# identify the dominant forest species for each "tm_id"
spp_dominance = (
    grid_tree_r.groupby(['tm_id', 'ForestType'])['TREE']  # Sum tree counts (or use another metric)
    .sum()
    .reset_index()
    .sort_values(['tm_id', 'TREE'], ascending=[True, False])
    .drop_duplicates('tm_id')  # Keep only the top species per tm_id
    .rename(columns={'ForestType': 'MajoritySpp'})
)

# join back to the tm_id summary
grid_tmid_spp = grid_tmid.merge(spp_dominance[['tm_id', 'MajoritySpp']], on='tm_id', how='left')

# identify the landscape proportion of dominant species
spp_pr = (
    grid_tmid_spp.groupby('MajoritySpp')['count']
    .sum()
    .reset_index()
    .rename(columns={'count': 'maj_spp_count'})
    .sort_values(by='maj_spp_count', ascending=False)
)

# calculate the fraction
spp_pr['fraction'] = spp_pr['maj_spp_count'] / spp_pr['maj_spp_count'].sum()
spp_pr = spp_pr.sort_values(by='fraction', ascending=False)

# Identify species contributing 97% of the burned area
spp_pr['c_fraction'] = spp_pr['fraction'].cumsum()
top_species = spp_pr[spp_pr['c_fraction'] <= 0.97]
print(f"\nSpecies contributing to 97% of the burned area:\n{top_species}\n")

In [ ]:
spp_pct_cover.head()

In [ ]:
# test tree table
tree_tbl[tree_tbl['tm_id'] == 21404][['tm_id','COMMON_NAME','DIA','HT','CR','TREE']]